In [1]:
#Directory Structure
#~/Documents/TAQData <--- has the parquet files
#~/gitcode/AmitG/TAQCode <--- has the jupypter and python code


In [2]:
import pyarrow.parquet as pq

# ---- quotes file ----
quotes_path = '~/Documents/TAQData/data_quotes_2023_12_15.parquet'
pf_quotes = pq.ParquetFile(quotes_path)
print("🧾 Quotes schema:")
print(pf_quotes.schema_arrow)     # column names and types only

# ---- trades file ----
trades_path = '~/Documents/TAQData/data_trades_2023_12_15.parquet'
pf_trades = pq.ParquetFile(trades_path)
print("\n💹 Trades schema:")
print(pf_trades.schema_arrow)


🧾 Quotes schema:
DATE: date32[day]
TIME_M: time64[ns]
EX: large_string
BID: double
BIDSIZ: int64
ASK: double
ASKSIZ: int64
QU_COND: large_string
QU_SEQNUM: int64
NATBBO_IND: large_string
QU_CANCEL: large_string
QU_SOURCE: large_string
SYM_ROOT: large_string
SYM_SUFFIX: large_string

💹 Trades schema:
date: date32[day]
time_m: time64[us]
time_m_nano: int16
ex: string
sym_root: string
sym_suffix: string
tr_scond: string
size: int32
price: double
tr_stop_ind: string
tr_corr: string
tr_seqnum: int64
tr_id: string
tr_source: string
tr_rf: string
part_time: time64[us]
part_time_nano: int16
trf_time: time64[us]
trf_time_nano: int16
tte_ind: string


In [3]:
import pyarrow.parquet as pq

# Paths to your files
quotes_path = '~/Documents/TAQData/data_quotes_2023_12_15.parquet'
trades_path = '~/Documents/TAQData/data_trades_2023_12_15.parquet'

# Efficiently count rows without loading data
quotes_rows = pq.ParquetFile(quotes_path).metadata.num_rows
trades_rows = pq.ParquetFile(trades_path).metadata.num_rows

print(f"Quotes rows: {quotes_rows:,}")
print(f"Trades rows: {trades_rows:,}")


Quotes rows: 1,957,290,808
Trades rows: 89,518,138


In [4]:
import pyarrow.parquet as pq
import os

DATA_DIR = "~/Documents/TAQData"
quotes_path = os.path.expanduser(f"{DATA_DIR}/data_quotes_2023_12_15.parquet")
trades_path = os.path.expanduser(f"{DATA_DIR}/data_trades_2023_12_15.parquet")

def parquet_date_range(path, date_col):
    pf = pq.ParquetFile(path)
    for rg in range(pf.num_row_groups):
        df = pf.read_row_group(rg, columns=[date_col]).to_pandas()
        if not df.empty:
            dmin, dmax = df[date_col].min(), df[date_col].max()
            return dmin, dmax
    return None, None

quote_min, quote_max = parquet_date_range(quotes_path, "DATE")
trade_min, trade_max = parquet_date_range(trades_path, "date")

print(f"Quotes date range: {quote_min} → {quote_max}")
print(f"Trades date range: {trade_min} → {trade_max}")


Quotes date range: 2023-12-15 → 2023-12-15
Trades date range: 2023-12-15 → 2023-12-15


In [1]:
# 1) What symbols are actually present? (first ~20 row groups, TRADES)
import pyarrow.parquet as pq, pandas as pd, os
TRADES_FILE = os.path.expanduser("~/Documents/TAQData/data_trades_2023_12_15.parquet")
pf = pq.ParquetFile(TRADES_FILE)

seen = []
for rg in range(min(20, pf.num_row_groups)):
    df = pf.read_row_group(rg, columns=["sym_root","sym_suffix"]).to_pandas()
    df["sym_root"]  = df["sym_root"].astype(str).str.strip().str.upper()
    df["sym_suffix"]= df["sym_suffix"].astype(str).fillna("").str.strip().str.upper()
    seen.append(df.groupby(["sym_root","sym_suffix"]).size().reset_index(name="cnt"))
present = pd.concat(seen).groupby(["sym_root","sym_suffix"])["cnt"].sum().reset_index().sort_values("cnt", ascending=False)
print(present.head(25))


    sym_root sym_suffix     cnt
24      AAPL       NONE  625486
122     ADBE       NONE  140747
1         AA       NONE  128766
15       AAL       NONE   94629
52       ABT       NONE   77437
45      ABNB       NONE   76272
133      ADI       NONE   73992
32      ABBV       NONE   69560
165      AEE       NONE   68983
136      ADM       NONE   50396
91       ACN       NONE   48587
0          A       NONE   46061
75      ACGL       NONE   45061
150     ADSK       NONE   43176
180      AEO       NONE   42705
144      ADP       NONE   38899
176      AEM       NONE   35980
47       ABR       NONE   35705
19      AAOI       NONE   35294
10      AADI       NONE   32126
183      AEP       NONE   30936
21       AAP       NONE   28638
61      ACAD       NONE   26610
79      ACHR       NONE   26189
82       ACI       NONE   24420


In [4]:
#polar_try_partitioned.py for quotes and trades
import polars as pl
pl.Config.set_tbl_cols(50) 
pl.Config.set_tbl_width_chars(200)
#rename.py shall rename the cols of trades to uppercase and save since easier for joining

polars.config.Config

In [6]:
import polars as pl
import os

# Define the directory and pattern
directory = "../TAQData/processed_output_trades_upper"
pattern = os.path.join(directory, "chunk_*.parquet")

# Create a LazyFrame from all matching Parquet files
lazy_df = pl.scan_parquet(pattern)

# Access the schema (column names and types)
schema = lazy_df.collect_schema()

# Print the schema
print(schema)

Schema({'DATE': Date, 'TIME_M': Time, 'TIME_M_NANO': Int16, 'EX': String, 'SYM_ROOT': String, 'SYM_SUFFIX': String, 'TR_SCOND': String, 'SIZE': Int32, 'PRICE': Float64, 'TR_STOP_IND': String, 'TR_CORR': String, 'TR_SEQNUM': Int64, 'TR_ID': String, 'TR_SOURCE': String, 'TR_RF': String, 'PART_TIME': Time, 'PART_TIME_NANO': Int16, 'TRF_TIME': Time, 'TRF_TIME_NANO': Int16, 'TTE_IND': String})


In [14]:
print(lazy_df.filter(pl.col("SYM_ROOT") == "AAPL").head(10).collect())

shape: (10, 14)
┌────────────┬──────────────────┬─────┬────────┬───┬───────────┬───────────┬──────────┬────────────┐
│ DATE       ┆ TIME_M           ┆ EX  ┆ BID    ┆ … ┆ QU_CANCEL ┆ QU_SOURCE ┆ SYM_ROOT ┆ SYM_SUFFIX │
│ ---        ┆ ---              ┆ --- ┆ ---    ┆   ┆ ---       ┆ ---       ┆ ---      ┆ ---        │
│ date       ┆ time             ┆ str ┆ f64    ┆   ┆ str       ┆ str       ┆ str      ┆ str        │
╞════════════╪══════════════════╪═════╪════════╪═══╪═══════════╪═══════════╪══════════╪════════════╡
│ 2023-12-15 ┆ 03:59:00.1149253 ┆ Z   ┆ 0.0    ┆ … ┆ null      ┆ N         ┆ AAPL     ┆ null       │
│            ┆ 77               ┆     ┆        ┆   ┆           ┆           ┆          ┆            │
│ 2023-12-15 ┆ 03:59:00.1854122 ┆ K   ┆ 0.0    ┆ … ┆ null      ┆ N         ┆ AAPL     ┆ null       │
│            ┆ 94               ┆     ┆        ┆   ┆           ┆           ┆          ┆            │
│ 2023-12-15 ┆ 04:00:00.0044523 ┆ K   ┆ 197.53 ┆ … ┆ null      ┆ N         

In [7]:
import polars as pl
import os

# Define the directory and pattern
directory = "../TAQData/processed_output_quotes"
pattern = os.path.join(directory, "chunk_*.parquet")

# Create a LazyFrame from all matching Parquet files
lazy_df = pl.scan_parquet(pattern)

# Access the schema (column names and types)
schema = lazy_df.collect_schema()

# Print the schema
print(schema)

Schema({'DATE': Date, 'TIME_M': Time, 'EX': String, 'BID': Float64, 'BIDSIZ': Int64, 'ASK': Float64, 'ASKSIZ': Int64, 'QU_COND': String, 'QU_SEQNUM': Int64, 'NATBBO_IND': String, 'QU_CANCEL': String, 'QU_SOURCE': String, 'SYM_ROOT': String, 'SYM_SUFFIX': String})


In [8]:
print(lazy_df.filter((pl.col('SYM_ROOT') == 'AAPL') & ( pl.col('BIDSIZ').is_not_null() )).head(10).collect() )

shape: (10, 14)
┌────────────┬────────────────────┬─────┬────────┬────────┬────────┬────────┬─────────┬───────────┬────────────┬───────────┬───────────┬──────────┬────────────┐
│ DATE       ┆ TIME_M             ┆ EX  ┆ BID    ┆ BIDSIZ ┆ ASK    ┆ ASKSIZ ┆ QU_COND ┆ QU_SEQNUM ┆ NATBBO_IND ┆ QU_CANCEL ┆ QU_SOURCE ┆ SYM_ROOT ┆ SYM_SUFFIX │
│ ---        ┆ ---                ┆ --- ┆ ---    ┆ ---    ┆ ---    ┆ ---    ┆ ---     ┆ ---       ┆ ---        ┆ ---       ┆ ---       ┆ ---      ┆ ---        │
│ date       ┆ time               ┆ str ┆ f64    ┆ i64    ┆ f64    ┆ i64    ┆ str     ┆ i64       ┆ str        ┆ str       ┆ str       ┆ str      ┆ str        │
╞════════════╪════════════════════╪═════╪════════╪════════╪════════╪════════╪═════════╪═══════════╪════════════╪═══════════╪═══════════╪══════════╪════════════╡
│ 2023-12-15 ┆ 03:59:00.114925377 ┆ Z   ┆ 0.0    ┆ 0      ┆ 0.0    ┆ 0      ┆ L       ┆ 1103      ┆ 1          ┆ null      ┆ N         ┆ AAPL     ┆ null       │
│ 2023-12-15 ┆ 03:

In [19]:
#run query.py for joint dataset